# Anwendung von maschinellem Lernen auf den KHK_Klassifikation.csv Datensatz

## Praktische Demonstration für verschiedene machine Learning Modelle

### Tim Bleicher, Linus Pfeifer

Dieses Jupyter Notebook demonstriert die Anwendung von verschiedenen Machine Learning Modellen auf den KHK_Klassifikation.csv Datensatz. 

**Inhaltsverzeichnis:**
- [1. Einbindung der Daten](#1-einbindung-der-daten)
  - [1.1 explorative Analyse der Daten](#11-explorative-analyse-der-daten)
- [2. PCA-Dimensionsreduzierung zur Visualisierung und Analyse der Daten](#2-pca-dimensionsreduzierung-zur-visualisierung-und-analyse-der-daten)
  - [Funktionsweise von PCA](#funktionsweise-von-pca)
  - [Lässt sich aus den PCA-Daten eine potentielle gute Separierbarkeit der Klassen ablesen?](#lässt-sich-aus-den-pca-daten-eine-potentielle-gute-separierbarkeit-der-klassen-ablesen)
- [3. Anwendung verschiedener vorgestellter Klassifikationsverfahren](#3-anwendung-verschiedener-vorgestellter-klassifikationsverfahren)
  - [Definition und Datenvorbereitung](#definition-und-datenvorbereitung)
  - [3.1 logistische Regression](#31-logistische-regression)
    - [Modell definieren und trainieren](#modell-definieren-und-trainieren)
    - [Modell testen](#modell-testen)
  - [3.2 Entscheidungsbäume](#32-entscheidungsbäume)
    - [3.2.1 klassische Entscheidungsbäume](#321-klassische-entscheidungsbäume)
    - [3.2.2 Bagging in Form von Random Forest](#322-bagging-in-form-von-random-forest)
    - [3.2.3 Boosting in Form von AdaBoost](#323-boosting-in-form-von-adaboost)
    - [3.2.4 Stacking](#324-stacking)
  - [3.3 k-Nearest-Neighbor](#33-k-nearest-neighbor)
    - [3.3.1 k-Nearest-Neighbor mit euklidischer Metrik](#331-k-nearest-neighbor-mit-euklidischer-metrik)
    - [3.3.2 k-Nearest-Neighbor mit manhattan Metrik](#332-k-nearest-neighbor-mit-manhattan-metrik)
    - [3.3.4 k-Nearest-Neighbor mit Minkowski Metrik und p = 3](#334-k-nearest-neighbor-mit-minkowski-metrik-und-p--3)
  - [3.4 Support Vector Machine](#34-support-vector-machine)
  - [3.5 Neuronales Netz](#35-neuronales-netz)

## 1. Einbindung der Daten

Zu beginn des Projekts werden die Daten zunächst geladen um diese im anschluss analysieren und nutzen zu können.

In [ ]:
pip install -r requirements.txt

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
data = pd.read_csv('KHK_Klassifikation.csv', sep=',')

In [ ]:
print(data.head())

### 1.1 explorative Analyse der Daten 
TODO

## 2. PCA-Dimensionsreduzierung zur Visualisierung und Analyse der Daten 

### Funktionsweise von PCA
Die Hauptkomponentenanalyse (PCA) dient der Dimensionsreduktion eines Datensatzes. Dies ermöglicht beispielsweise verschiedene Analyse des gesamten Datensatzes (mit mehr als 3 Dimensionen), wobei die Ergebnisse durch die Dimensionsreduktion weiterhin visualisiert werden können.
Das Verfahren der PCA läuft nach folgendem Schema ab:

1. Berechnung des Mittelwerts und Zentrierung der Daten
2. Berechnung der Kovarianzmatrix
3. Berechnung der Eigenwerte und Eigenvektoren
4. Transformation der Daten

Damit die PCA korrekt funktioniert, muss zunächst von jeder Dimension der Mittelwert subtrahiert werden. Dieser Mittelwert entspricht dem Durchschnittswert jeder Dimension. Beispielsweise wird von allen $x$-Werten der Mittelwert $\overline{x}$ subtrahiert. Entsprechendes gilt für die anderen Dimensionen der Daten. Dadurch entsteht ein Datensatz mit einem Mittelwert von null.

Im nächsten Schritt wird die Kovarianzmatrix berechnet, welche die wechselseitigen Zusammenhänge zwischen den Merkmalen quantifiziert. Falls zwei Merkmale stark korrelieren, können diese in einer neuen Achse kombiniert werden.

Anschließend werden die Eigenwerte und Eigenvektoren der Kovarianzmatrix bestimmt. Die Eigenvektoren definieren die Richtungen der Hauptkomponenten, während die zugehörigen Eigenwerte die Bedeutung bzw. die Varianz der jeweiligen Eigenvektoren widerspiegeln.

Es folgt die eigentliche Dimensionsreduktion, indem nur diejenigen Eigenvektoren mit den größten Eigenwerten ausgewählt werden. Diese Eigenvektoren entsprechen den neuen Hauptachsen des Datensatzes.

Schließlich werden die Daten transformiert, indem die ursprüngliche Datenmatrix mit der Matrix der Eigenvektoren multipliziert wird. In dieser Matrix repräsentiert jede Spalte einen Eigenvektor.



In [7]:
label_encoder = LabelEncoder()
categorical_columns = ['Geschlecht', 'EKG', 'AP']
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])
print(data.head())

   Alter  Geschlecht  Blutdruck  Chol  Blutzucker  EKG  HFmax  AP   RZ  KHK
0     40           1        140   289           0    1    172   0  0.0    0
1     49           0        160   180           0    1    156   0  1.0    1
2     37           1        130   283           0    2     98   0  0.0    0
3     48           0        138   214           0    1    108   1  1.5    1
4     54           1        150   195           0    1    122   0  0.0    0


In [8]:
# Entferne die Zielvariable "KHK" vor der Skalierung
data_without_target = data.drop(columns=["KHK"], errors="ignore")

# Skalierung der Daten
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_without_target)

# PCA-Transformation mit zwei Hauptkomponenten
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data_scaled)

# Umwandlung der PCA-Ergebnisse in einen DataFrame
df_pca = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])

# Interaktive Visualisierung
fig = px.scatter(df_pca, x='PC1', y='PC2', title='PCA-Visualisierung der Daten', opacity=0.5)
fig.show()


### Lässt sich aus den PCA-Daten eine potentielle gute Separierbarkeit der Klassen ablesen? 
TODO
--> Ich würde sagen nein, lass aber mal drüber quatschen 

## 3. Anwendung verschiedener vorgestellter Klassifikationsverfahren

#### Definition und Datenvorbereitung

In [28]:
# Kategorische und numerische Spalten definieren
categorical_features = ["Geschlecht", "EKG", "AP"]
numerical_features = ["Alter", "Blutdruck", "Chol", "Blutzucker", "HFmax", "RZ"]

# Zielvariable und Features auswählen
# Explizite Kopie erstellen, um die Warnung zu vermeiden
X = data[categorical_features + numerical_features].copy()
y = data["KHK"]

# Features in der gleichen Reihenfolge wie im PCA-Histogramm anordnen
desired_order = ["Alter", "Geschlecht", "Blutdruck", "Chol", "Blutzucker", "EKG", "HFmax", "AP", "RZ"]
X = X[desired_order]

# Label-Encoding für kategorische Variablen
label_encoders = {}  # Dictionary, um die LabelEncoder-Objekte zu speichern
for col in categorical_features:
    label_encoders[col] = LabelEncoder()
    X[col] = label_encoders[col].fit_transform(X[col])

# Standardisierung für numerische Variablen
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 3.1 logistische Regression 

#### Modell definieren und trainieren

In [ ]:
# Logistic Regression für binäre Klassifikation

# Pipeline mit Vorverarbeitung und logistische Regression
model = LogisticRegression()
# Modell trainieren
model.fit(X_train, y_train)

#### Modell testen

In [ ]:
# Vorhersagen treffen
y_pred_log_reg = model.predict(X_test)

# Evaluierung
accuracy = accuracy_score(y_test, y_pred_log_reg)
classification_rep = classification_report(y_test, y_pred_log_reg)

print(f"Accuracy: {accuracy:.2f}")
print(classification_rep)

### 3.2 Entscheidungsbäume

#### 3.2.1 klassische Entscheidungsbäume

In [ ]:
clf_tree = DecisionTreeClassifier(random_state=42)
clf_tree.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_tree = clf_tree.predict(X_test)

# Genauigkeit berechnen
accuracy_tree = accuracy_score(y_test, y_pred_tree)
classification_rep_tree = classification_report(y_test, y_pred_tree)
print(f"Modellgenauigkeit: {accuracy_tree:.2f}")
print(classification_rep_tree)

#### 3.2.2 Bagging in Form von Random Forest

In [ ]:
# Random Forest Modell trainieren
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_random_forest = clf.predict(X_test)

# Genauigkeit berechnen
accuracy_random_forest = accuracy_score(y_test, y_pred_random_forest)
classification_rep = classification_report(y_test, y_pred_random_forest)
print(f"Modellgenauigkeit: {accuracy_random_forest:.2f}")
print(classification_rep)

Modellgenauigkeit: 0.73
              precision    recall  f1-score   support

           0       0.68      0.70      0.69        77
           1       0.78      0.76      0.77       107

    accuracy                           0.73       184
   macro avg       0.73      0.73      0.73       184
weighted avg       0.74      0.73      0.73       184



#### 3.2.3 Boosting in Form von AdaBoost

In [ ]:
base_estimator = DecisionTreeClassifier(max_depth=1)
adaboost_model = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=50,
    learning_rate=0.3,
    random_state=42
)

adaboost_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_ada = adaboost_model.predict(X_test)

# Genauigkeit berechnen
accuracy_random_forest = accuracy_score(y_test, y_pred_ada)
classification_rep = classification_report(y_test, y_pred_ada)
print(f"Modellgenauigkeit: {accuracy_random_forest:.2f}")
print(classification_rep)

#### 3.2.4 Stacking

In [ ]:
# Basismodelle: KNN, SVM und Logistische Regression
base_estimators = [
    ('knn', KNeighborsClassifier(n_neighbors=5)),  # KNN mit 5 Nachbarn
    ('svc', SVC(kernel='linear', random_state=42)),  # SVM mit linearem Kernel
    ('logreg', LogisticRegression(random_state=42))  # Logistische Regression
]

# Finales Modell (Meta-Modell)
final_estimator = LogisticRegression()

# StackingClassifier erstellen
stacking_model = StackingClassifier(estimators=base_estimators, final_estimator=final_estimator)

# Modell trainieren
stacking_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_stack = stacking_model.predict(X_test)

# Genauigkeit berechnen
accuracy_stack = accuracy_score(y_test, y_pred_stack)
classification_rep = classification_report(y_test, y_pred_stack)

print(f"Modellgenauigkeit: {accuracy_stack:.2f}")
print(classification_rep)

### 3.3 k-Nearest-Neighbor

#### 3.3.1 k-Nearest-Neighbor mit euklidischer Metrik

In [ ]:
# k-NN Modell mit k=10 erstellen
knn_model = KNeighborsClassifier(n_neighbors=10, metric='euclidean')

# Modell trainieren
knn_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_knn = knn_model.predict(X_test)

# Genauigkeit berechnen
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_rep_knn = classification_report(y_test, y_pred_knn)

print(f"Modellgenauigkeit: {accuracy_knn:.2f}")
print(classification_rep_knn)


#### 3.3.2 k-Nearest-Neighbor mit manhattan Metrik

In [ ]:
# k-NN Modell mit k=10 erstellen
knn_model = KNeighborsClassifier(n_neighbors=10, metric='manhattan')

# Modell trainieren
knn_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_knn = knn_model.predict(X_test)

# Genauigkeit berechnen
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_rep_knn = classification_report(y_test, y_pred_knn)

print(f"Modellgenauigkeit: {accuracy_knn:.2f}")
print(classification_rep_knn)


#### 3.3.4 k-Nearest-Neighbor mit Minkowski Metrik und p = 3

In [ ]:
# k-NN Modell mit k=10 erstellen
knn_model = KNeighborsClassifier(n_neighbors=10, metric='minkowski', p=3)

# Modell trainieren
knn_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_knn = knn_model.predict(X_test)

# Genauigkeit berechnen
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_rep_knn = classification_report(y_test, y_pred_knn)

print(f"Modellgenauigkeit: {accuracy_knn:.2f}")
print(classification_rep_knn)


### 3.4 Support Vector Machine

In [ ]:
# k-NN Modell mit k=10 erstellen
svm_model = SVC(kernel='linear', random_state=42)

# Modell trainieren
svm_model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_svm = svm_model.predict(X_test)

# Genauigkeit berechnen
accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

print(f"Modellgenauigkeit: {accuracy_svm:.2f}")
print(classification_rep_svm)


### 3.5 Neuronales Netz

In [ ]:
def create_model(optimizer):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Modell kompilieren
    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

    # Modellübersicht anzeigen
    model.summary()
    return model

In [ ]:
sgd_model = create_model(optimizer='sgd')
# Modell trainieren
history_sgd = sgd_model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

# Modell evaluieren
test_loss_sgd, test_accuracy_sgd = sgd_model.evaluate(X_test, y_test)

# Trainingsverlauf visualisieren
plt.figure(figsize=(12, 4))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history_sgd.history['accuracy'], label='Training Accuracy')
plt.plot(history_sgd.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history_sgd.history['loss'], label='Training Loss')
plt.plot(history_sgd.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Vorhersagen machen
y_pred_sgd = sgd_model.predict(X_test)
y_pred_classes_sgd = (y_pred_sgd > 0.5).astype(int)

# Confusion Matrix und Classification Report
print(f"\nTest Accuracy: {test_accuracy_sgd:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes_sgd))

In [ ]:
adam_model = create_model(optimizer='adam')
# Modell trainieren
history_adam = adam_model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

# Modell evaluieren
test_loss, test_accuracy_adam = adam_model.evaluate(X_test, y_test)

# Trainingsverlauf visualisieren
plt.figure(figsize=(12, 4))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history_adam.history['accuracy'], label='Training Accuracy')
plt.plot(history_adam.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history_adam.history['loss'], label='Training Loss')
plt.plot(history_adam.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Vorhersagen machen
y_pred_adam = adam_model.predict(X_test)
y_pred_classes_adam = (y_pred_adam > 0.5).astype(int)

# Confusion Matrix und Classification Report
print(f"\nTest Accuracy: {test_accuracy_adam:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes_adam))

## 4. Bedeutung der einzelnen Features

### 4.1 Feature-Bedeutung von PCA

In [9]:
feature_names = data.columns.tolist()
feature_names.remove("KHK") 
# Bedeutung der ursprünglichen Features
feature_importance = np.abs(pca.components_).sum(axis=0)
# DataFrame für Plotly erstellen
df_plot = pd.DataFrame({"Feature": feature_names, "Wichtigkeit": feature_importance})

# Interaktive Visualisierung mit Plotly
fig = px.bar(df_plot, x="Feature", y="Wichtigkeit", title="Feature-Bedeutung aus PCA", labels={"Feature": "Feature", "Wichtigkeit": "Feature-Wichtigkeit"})
fig.update_xaxes() 
fig.show()

### 4.2 Feature-Bedeutung für Random Forest

In [ ]:
# Feature-Wichtigkeit aus Random Forest
feature_importance = clf.feature_importances_

# DataFrame für Plotly erstellen
df_plot = pd.DataFrame({"Feature": X.columns.tolist(), "Wichtigkeit": feature_importance})

# Interaktive Visualisierung mit Plotly
fig = px.bar(df_plot, x="Feature", y="Wichtigkeit", title="Feature-Bedeutung aus Random Forest", labels={"Feature": "Feature", "Wichtigkeit": "Feature-Wichtigkeit"})
fig.update_xaxes() 
fig.show()


[0.12961358 0.04301963 0.09403582 0.17491876 0.03218152 0.03011758
 0.15389165 0.18009136 0.16213011]
